# Homework 3

1. Python users, use Python's pandas library to read in the Stata-formatted dataset used in class called "SI Sales.dta".  R users, use R's foreign library to read in the Stata-formatted dataset called "SI Sales Old.dta".  Replicate all of the regression results for this dataset that I presented in class.  

2. There is an additional feature in this dataset called "sales_year", which captures the year the sale of a house in Staten Island occurred.  From this feature, generate a feature that is linear time trend.  (A linear time trend is a feature that takes on value "1" in the initial year and increments by "1" each subsequent year.  For example, if 2003 were "1", 2004 would be "2", 2005 would be "3", and so forth.)  Run a linear regression model that relates the sales price to unit size, land size, age, the Todt Hill indicator, and the linear time trend.  How would you interpret the estimated coefficient associated with the linear time trend?  What is the 95% confidence interval of your interpretation?  Based on your regression diagnostics, have you improved the fit of the house price sales data by including the linear time trend as an additional explanatory feature?  

3. As noted in class, the unit size and land size features are measured in squared meters.  Suppose I ask you to re-express these features using the Imperial system of square feet rather than square meters, but I express a concern that the interpretation of the estimated coefficients, such as age, would be changed.  Without acutally doing any statistical learning, what would you say to me about my concern?  Rerun the linear regression in 2. using the dwelling size and land size measured in square feet (rather than square meters).  What, if anything, has changed in your estimated coefficients?

4. (Challenging question.  Feel free to work together to the extent that it assists you.)  Assume the following data generating process (DGP) governs a random sample of size 10,000: $y_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \epsilon_i$  for $\epsilon_i \sim N(0,1)$.  Further assume for this DGP that  $\beta_0=\beta_1=\beta_2=1$.  (a) Suppose the following process governs your features: $x_{1i} \sim N(0,1)$ and $x_{2i} \sim N(0,1)$ are independent.  Using R or Python, calculate the correlation between features $x_{1i}$ and $x_{2i}$.  Mistakenly, you decide to estimate a linear regression that includes only the feature $x_{1i}$.  Using R or Python, simulate this DGP and run the mistaken linear regression that includes only feature $x_{1i}$.  What value do you obtain for the coefficient associated with with feature $x_{1i}$?  (b) Suppose instead that the follow process governs your features: $x_{1i}=z_i+\eta_i$ and $x_{2i}=-z_i+\omega_i$, where $z_i \sim N(0,1)$, $\eta_i \sim N(0,1)$, and $\omega_i \sim N(0,1)$ are independent.  Using R or Python, calculate the correlation between features $x_{1i}$ and $x_{2i}$.  Again, you mistakenly decide to estimate a linear regression that includes only the feature $x_{1i}$.  Using R or Python, simulate this DGP and run the mistaken linear regression that includes only feature $x_{1i}$.  What value do you obtain for the coefficient associated with with feature $x_{1i}$?  (c) Are there any conclusions you can draw from your results in (a) and (b)? 

4.  Spend time working on your proposal for the Foundations Project.

For questions 1 through 4 above, submit code and results.

In [8]:
# Load relevant libraries.

%pylab inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import Quandl
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.api import abline_plot
import patsy
import seaborn as sns
sns.set(context='notebook', style='whitegrid', palette='deep', font='sans-serif', font_scale=1, rc=None)
import urllib2 as url

Populating the interactive namespace from numpy and matplotlib


In [3]:
sales = pd.read_stata('SI Sales Old.dta')

In [4]:
print sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31680 entries, 0 to 31679
Data columns (total 6 columns):
price         31680 non-null float64
unit_size     31680 non-null float32
land_size     31680 non-null float32
age           31680 non-null float32
todt          31680 non-null float32
sales_year    31680 non-null float32
dtypes: float32(5), float64(1)
memory usage: 1.1 MB
None


In [5]:
print sales.describe()

                price     unit_size     land_size           age          todt  \
count    31680.000000  31680.000000  31680.000000  31680.000000  31680.000000   
mean    402106.300221    161.039062    326.058075     35.846844      0.007102   
std     239551.524308     79.404953    289.826691     31.082787      0.083977   
min      50500.000000     20.067047     24.340586      0.000000      0.000000   
25%     291975.000000    114.456497    185.806000      9.000000      0.000000   
50%     371000.000000    145.671906    252.696167     27.000000      0.000000   
75%     455000.000000    184.969879    380.902313     56.000000      0.000000   
max    7700000.000000   1431.077759  10788.825195    119.000000      1.000000   

         sales_year  
count  31680.000000  
mean    2006.483643  
std        2.785286  
min     2003.000000  
25%     2004.000000  
50%     2006.000000  
75%     2009.000000  
max     2013.000000  


In [6]:
print sales.corr()

               price  unit_size  land_size       age      todt  sales_year
price       1.000000   0.549706   0.505356 -0.064088  0.352151    0.096092
unit_size   0.549706   1.000000   0.446105 -0.201536  0.326307    0.023416
land_size   0.505356   0.446105   1.000000  0.274697  0.346801    0.077402
age        -0.064088  -0.201536   0.274697  1.000000  0.005157    0.125040
todt        0.352151   0.326307   0.346801  0.005157  1.000000    0.012260
sales_year  0.096092   0.023416   0.077402  0.125040  0.012260    1.000000


In [9]:
mod = smf.ols(formula='price ~ unit_size', data = sales).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                 1.372e+04
Date:                Wed, 30 Sep 2015   Prob (F-statistic):               0.00
Time:                        17:49:12   Log-Likelihood:            -4.3166e+05
No. Observations:               31680   AIC:                         8.633e+05
Df Residuals:                   31678   BIC:                         8.633e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    1.35e+05   2542.335     53.118      0.0

In [10]:
mod.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Results: Ordinary least squares
=======================================================================
Model:               OLS               Adj. R-squared:      0.302      
Dependent Variable:  price             AIC:                 863318.9733
Date:                2015-09-30 17:49  BIC:                 863335.7002
No. Observations:    31680             Log-Likelihood:      -4.3166e+05
Df Model:            1                 F-statistic:         1.372e+04  
Df Residuals:        31678             Prob (F-statistic):  0.00       
R-squared:           0.302             Scale:               4.0046e+10 
-----------------------------------------------------------------------
             Coef.     Std.Err.    t     P>|t|     [0.025      0.975]  
-----------------------------------------------------------------------
Intercept 135043.8679 2542.3346  53.1181 0.0000 130060.7932 140026.9425
unit_size   1658.3706   14.1594 117.1215 0.0000   1630.6176   1686.1236
-----------------------------------------------------------------------
Omnibus:            51914.230      Durbin-Watson:         1.973        
Prob(Omnibus):      0.000          Jarque-Bera (JB):      103733692.864
Skew:               10.534         Prob(JB):              0.000        
Kurtosis:           282.539        Condition No.:         406          
=======================================================================

"""

In [11]:
mod = smf.ols(formula='price ~ unit_size + land_size', data = sales).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     9985.
Date:                Wed, 30 Sep 2015   Prob (F-statistic):               0.00
Time:                        17:49:54   Log-Likelihood:            -4.2961e+05
No. Observations:               31680   AIC:                         8.592e+05
Df Residuals:                   31677   BIC:                         8.593e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1.179e+05   2397.598     49.177      0.0

In [12]:
mod = smf.ols(formula='price ~ unit_size + land_size + age', data = sales).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     6840.
Date:                Wed, 30 Sep 2015   Prob (F-statistic):               0.00
Time:                        17:50:11   Log-Likelihood:            -4.2945e+05
No. Observations:               31680   AIC:                         8.589e+05
Df Residuals:                   31676   BIC:                         8.589e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1.496e+05   2940.848     50.854      0.0

In [13]:
mod = smf.ols(formula='price ~ unit_size + land_size + age + todt', data = sales).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     5421.
Date:                Wed, 30 Sep 2015   Prob (F-statistic):               0.00
Time:                        17:50:30   Log-Likelihood:            -4.2910e+05
No. Observations:               31680   AIC:                         8.582e+05
Df Residuals:                   31675   BIC:                         8.582e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1.674e+05   2985.210     56.076      0.0